In [1]:
# Install pandas library

%pip install pandas

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install -U pip setuptools wheel

   ---------------------------------------- 0.0/1.8 MB ? eta -:--:--
    --------------------------------------- 0.0/1.8 MB 991.0 kB/s eta 0:00:02
   ----- ---------------------------------- 0.2/1.8 MB 3.0 MB/s eta 0:00:01
   ---------------- ----------------------- 0.7/1.8 MB 6.7 MB/s eta 0:00:01
   ---------------------------- ----------- 1.3/1.8 MB 8.2 MB/s eta 0:00:01
   ---------------------------------------- 1.8/1.8 MB 9.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ------------------- -------------------- 0.6/1.2 MB 19.2 MB/s eta 0:00:01
   -------------------------------------- - 1.2/1.2 MB 15.1 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 15.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/72.5 kB ? eta -:--:--
   ---------------------------------------- 72.5/72.5 kB ? eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Succe

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [5]:
#Data loading/ Data manipulation
import pandas as pd
import numpy as np
import spacy
from spacy.training import Example
import random


In [15]:
# Load dataset
file_path = 'assets/dataset/Resume.csv'
df = pd.read_csv(file_path)
df

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR
...,...,...,...,...
2479,99416532,RANK: SGT/E-5 NON- COMMISSIONED OFFIC...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2480,24589765,"GOVERNMENT RELATIONS, COMMUNICATIONS ...","<div class=""fontsize fontface vmargins hmargin...",AVIATION
2481,31605080,GEEK SQUAD AGENT Professional...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION
2482,21190805,PROGRAM DIRECTOR / OFFICE MANAGER ...,"<div class=""fontsize fontface vmargins hmargin...",AVIATION


In [19]:
def clean_annotations(text, entities):
    """Ensure that entity spans are clean and do not include leading/trailing spaces."""
    cleaned_entities = []
    for start, end, label in entities:
        # Trim leading/trailing whitespace
        while start < len(text) and text[start].isspace():
            start += 1
        while end > 0 and text[end - 1].isspace():
            end -= 1
        # Append only if valid
        if start < end:
            cleaned_entities.append((start, end, label))
    return cleaned_entities

# Modify the TRAIN_DATA to clean entities
TRAIN_DATA = []
for index, row in df.iterrows():
    text = row['Resume_str']
    label = row['Category']
    entities = [(0, len(text), "JOB_CATEGORY")]  # Modify this according to your actual needs
    cleaned_entities = clean_annotations(text, entities)
    if cleaned_entities:
        TRAIN_DATA.append((text, {"entities": cleaned_entities}))


In [20]:
# Create a blank spaCy model
nlp = spacy.blank("en")

In [21]:
# Add the NER pipeline if not present
if "ner" not in nlp.pipe_names:
    ner = nlp.add_pipe("ner", last=True)
else:
    ner = nlp.get_pipe("ner")

# Add label to NER
ner.add_label("JOB_CATEGORY")

1

In [22]:
# Train the NER model
optimizer = nlp.begin_training()
for i in range(10):  # Number of iterations
    random.shuffle(TRAIN_DATA)
    losses = {}
    for text, annotations in TRAIN_DATA:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], drop=0.5, losses=losses)
    print(f"Iteration {i} - Losses: {losses}")

Iteration 0 - Losses: {'ner': np.float32(10839.023)}
Iteration 1 - Losses: {'ner': np.float32(129.28738)}
Iteration 2 - Losses: {'ner': np.float32(32.796448)}
Iteration 3 - Losses: {'ner': np.float32(41.298805)}
Iteration 4 - Losses: {'ner': np.float32(36.95062)}
Iteration 5 - Losses: {'ner': np.float32(25.061316)}
Iteration 6 - Losses: {'ner': np.float32(30.35422)}
Iteration 7 - Losses: {'ner': np.float32(19.443224)}
Iteration 8 - Losses: {'ner': np.float32(20.267483)}
Iteration 9 - Losses: {'ner': np.float32(7.1316586)}


In [23]:
# Save the trained model
nlp.to_disk("models")
print("Model training completed and saved!")

Model training completed and saved!
